In [2]:
import search_engine
import math
from search_engine import *
import pickle
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from collections import defaultdict
import example_pos
from example_pos import *

In [3]:
#original index-contains just the abstract
# index_v2 contains the abstract introduction and conclusion
# 
f = open('index_title_abs','rb')
index = pickle.load(f)
f.close()

query = 'review help suggest'
# Retrieve will return a ranked list of tuples of the form (documentID, documentScore).
print(Retrieve(query,index,1000,'nofeedback',0.1,10,50,0.9)[0:10])

[('P11-2088', 1.560755882714912), ('N13-2011', 1.5568653873920246), ('W06-1650', 1.4811053903847555), ('W11-1402', 1.3347899903364284), ('W12-0608', 1.2852938587875227), ('W10-0207', 1.2849812980588509), ('J11-1014', 1.1836936383180425), ('C10-2035', 1.1508444478262378), ('W10-1205', 1.1271211985155167), ('W13-5008', 1.0433620928310816)]


In [4]:
data = pickle.load(open('AllDataPickle_e1.pk','rb'))
len(data)

20399

In [5]:
stopwords = nltk.corpus.stopwords.words('english')+[',','.','!','``',"''",'?',"'s",';','$',':',"'",'_', ')', '(']
expanded_indicators = ['futur', 'work', 'plan', 'improv', 'explor', 'approach', 'perform', 'research', 'evalu', 'extend', 'would']
stemmer = PorterStemmer()
docCollection = defaultdict(list)
termFrequency = defaultdict(int)
idf = defaultdict(int)
docIndex = defaultdict(list)
tfidf = defaultdict(int)
numberofwords = 0
docnumberofwords = defaultdict(int)
numOfDocuments = 0
#TF-IDF scores of the documents
for key in data:
    future_work_section = data[key][9]
    if not (future_work_section == ""):
        numOfDocuments = numOfDocuments + 1
        for token in [t.lower() for t in nltk.word_tokenize(future_work_section)]:
            if token in stopwords:
                continue
            if stemmer:
                token = stemmer.stem(token)
            if key not in docCollection:
                docCollection[key] = []

            docCollection[key].append(token)

            if token not in docIndex:
                docIndex[token] = defaultdict(int)

            if key not in docIndex[token]:
                docIndex[token][key] = 1

            else:
                docIndex[token][key] += 1

            numberofwords += 1
            docnumberofwords[key] += 1

            
for word in docIndex:
    sumword = 0
    for key in docIndex[word]:
        sumword = sumword + docIndex[word][key]
    termFrequency[word] = sumword/(numberofwords)
    idf[word] = 6*log ((numOfDocuments)/len(docIndex[word]), 10)
    tfidf[word] = termFrequency[word]*idf[word]

In [14]:
docCollection?

In [55]:
print (termFrequency['work'])
print (len(index['work']))
print (idf['work'])
print (tfidf['work'])


0.04051350257329555
3116
1.3821031645867
0.055993840115043186


In [6]:
[ item[0]  for item in sorted(termFrequency.items(),key=lambda k_v: k_v[1],reverse=True)][0:50]

['use',
 'perform',
 'approach',
 'improv',
 'work',
 'evalu',
 'model',
 'system',
 'would',
 'futur',
 'result',
 'method',
 'also',
 'featur',
 'research',
 'languag',
 'data',
 'word',
 'show',
 'gener',
 'differ',
 'task',
 'set',
 'plan',
 'inform',
 'translat',
 'train',
 '%',
 'present',
 'semant',
 'base',
 'explor',
 'experi',
 'learn',
 'paper',
 'extend',
 'algorithm',
 'two',
 'better',
 'test',
 'like',
 'compar',
 'one',
 'propos',
 'sentenc',
 'pars',
 'et',
 'combin',
 'extract',
 'text']

In [9]:
[(word,score) for (word,score) in sorted(termFrequency.items(),key=lambda k_v: k_v[1],reverse=True)[0:100] if word in expanded_indicators]

[('perform', 0.013544264549265965),
 ('approach', 0.012673398131546665),
 ('improv', 0.012669353860256947),
 ('work', 0.012593860796182207),
 ('evalu', 0.009618625217379582),
 ('would', 0.007516952237156069),
 ('futur', 0.0068820016446703245),
 ('research', 0.00494075142560563),
 ('plan', 0.003901373704148074),
 ('explor', 0.003074994270615673),
 ('extend', 0.002926704323326009)]

In [8]:
sorted(idf.items(),key=lambda k_v: k_v[1],reverse=True)[0:100]

[('creature.', 22.568860677268212),
 ('ntailment', 22.568860677268212),
 ('movementyand', 22.568860677268212),
 ('van', 22.568860677268212),
 ('stars.si', 22.568860677268212),
 ('mappings.w', 22.568860677268212),
 ('abus', 22.568860677268212),
 ('decision-mak', 22.568860677268212),
 ('dcc', 22.568860677268212),
 ('tasks.exploit', 22.568860677268212),
 ('coregress', 22.568860677268212),
 ('poems.for', 22.568860677268212),
 ('21.98', 22.568860677268212),
 ('surface-ori', 22.568860677268212),
 ('quantum', 22.568860677268212),
 ('industri', 22.568860677268212),
 ('mac', 22.568860677268212),
 ('0.0716', 22.568860677268212),
 ('dvai', 22.568860677268212),
 ('lexicalization.thu', 22.568860677268212),
 ('method.futur', 22.568860677268212),
 ('23.04', 22.568860677268212),
 ('tree-to-str', 22.568860677268212),
 ('saraclar', 22.568860677268212),
 ('sensevla-3', 22.568860677268212),
 ('delv', 22.568860677268212),
 ('multi-resolut', 22.568860677268212),
 ('-p', 22.568860677268212),
 ('pf-ccg', 22.5

In [9]:
sorted(tfidf.items(),key=lambda k_v: k_v[1],reverse=True)

[('plan', 0.07037579118942662),
 ('use', 0.05293213456402674),
 ('work', 0.049383936760795),
 ('model', 0.048664444356588656),
 ('system', 0.04700042397861948),
 ('improv', 0.0401398746472039),
 ('method', 0.039738644086697625),
 ('featur', 0.03859161667116897),
 ('also', 0.037828175741242066),
 ('investig', 0.0367591223663174),
 ('languag', 0.03574565202541156),
 ('includ', 0.03491143494268731),
 ('research', 0.034785915219210935),
 ('explor', 0.034313520153282755),
 ('evalu', 0.03264952383810066),
 ('data', 0.03237567871299978),
 ('approach', 0.03137821494528966),
 ('inform', 0.031150493844951634),
 ('gener', 0.03066298925815121),
 ('perform', 0.030561586274568946),
 ('would', 0.03030543606133991),
 ('word', 0.030114646538643743),
 ('direct', 0.028941467184662204),
 ('differ', 0.02891799834367409),
 ('task', 0.02837616561175874),
 ('extend', 0.027787520134415027),
 ('result', 0.02763279458977985),
 ('semant', 0.026808557251832787),
 ('develop', 0.02642552375127551),
 ('appli', 0.0262

In [13]:
data_keys = data.keys()
fw_keys = []
num_FW = 0
last_nonKey = ""
for i, key in enumerate(data_keys):
    curr_FW = data[key][-1]
    if not (curr_FW == ""):
        #print (i,curr_FW)
        num_FW = num_FW + 1
        fw_keys.append(key)
    else:
        last_nonKey = key
    

In [14]:
len(fw_keys)

12554

In [12]:
def get_score(data, results):
    dat_score = 0
    t_c = 2014
    rank = 0
    denom = 0
    topK = 10
    avg_year = 0
    avg_score = 0
    for (key, rel_score) in results[0:topK]:
        rank = rank + 1;
        if (rank == 1):
            denom = rel_score
        t_d = data[key][2]
        if (t_c == t_d) or (rel_score/denom < 0.01):
            print (key, rel_score/denom)
        dat_score = dat_score + exp(t_d - t_c)*(rel_score/denom)
        avg_score = rel_score/denom
        avg_year = avg_year + t_d
        #print(key + " " + str(t_d))
    print (avg_year/topK, avg_score/topK)
    return dat_score

In [27]:
#adds the citation of the
def get_score_citation(data, results, citations):
    dat_score = 0
    t_c = 2014
    rank = 0
    denom = 0
    topK = 10
    avg_year = 0
    avg_score = 0
    #sum from the search engine results 
    for (key, rel_score) in results[0:topK]:
        rank = rank + 1;
        if (rank == 1):
            denom = rel_score
        t_d = data[key][2]
        if (t_c == t_d) or (rel_score/denom < 0.01):
            print (key, rel_score/denom)
        dat_score = dat_score + exp(t_d - t_c)*(rel_score/denom)
        avg_score = rel_score/denom
        avg_year = avg_year + t_d
        #print(key + " " + str(t_d))
    
    #does not preserve order 
    result_dic = {}
    for (key,rel_score) in results:
        result_dic[key] = rel_score
        
    for c in citations:
        if c in result_dic:
            rel_score = result_dic[c]
            t_d = data[key][2]
            dat_score = dat_score + exp(t_d - t_c)*(rel_score/denom)
        
    print (avg_year/topK, avg_score/topK)
    return dat_score

In [ ]:
#adds the citation of the query plus the page rank
def get_score_citation_pr(data, results, citations, pr):
    dat_score = 0
    t_c = 2014
    K = 1000
    rank = 0
    denom = 0
    topK = 10
    avg_year = 0
    avg_score = 0
    #sum from the search engine results 
    for (key, rel_score) in results[0:topK]:
        rank = rank + 1;
        if (rank == 1):
            denom = rel_score
        t_d = data[key][2]
        if (t_c == t_d) or (rel_score/denom < 0.01):
            print (key, rel_score/denom)
        dat_score = dat_score + exp(t_d - t_c)*(rel_score/denom)
        avg_score = rel_score/denom
        avg_year = avg_year + t_d
        #print(key + " " + str(t_d))
    
    #does not preserve order 
    result_dic = {}
    for (key,rel_score) in results:
        result_dic[key] = rel_score
        
    for c in citations:
        if c in result_dic:
            rel_score = result_dic[c]
            t_d = data[key][2]
            dat_score = dat_score + exp(t_d - t_c - pr*K)*(rel_score/denom)
        
    print (avg_year/topK, avg_score/topK)
    return dat_score

In [26]:
lta_scores = []
indicators = ['futur', 'work', 'use', 'plan', 'model', 'improv', 'system', 'research', 'method', 'featur', 'includ', 'investig', 'explor', 'direct', 'languag', 'would', 'data', 'evalu', 'approach', 'perform']
uninformative_fw = 0
for key in fw_keys[:1]:
    print(key)
    future_work_section = data[key][9]
    abstract_section = data[key][6]
    query = future_work_section 
    print (query)
    phrases = " ".join(example_pos.get_phrases(query))
    clean_query = ""
    for item in phrases.split():
        if not( item in indicators):
            clean_query = clean_query + " " + item
    print (clean_query.strip())
    if not( clean_query.strip() == ""):
        result = Retrieve(clean_query.strip(),index,1000,'nofeedback',0.1,10,50,0.9)# [0:10]
        #lta_scores.append( (key,get_score(data,result) )  )
        #lta_scores.append( (key,get_score_citation(data, result, data[key][5]) )  )
        lta_scores.append( (key,get_score_citation(data, result, data[key][5], data[key][-1]) )  )
    else:
        uninformative_fw = uninformative_fw + 1

P12-1037
we have introduced a novel approach using bayesian logic programs to learn to infer implicit information from facts extracted from natural language text.our experimental evaluation on the ic data set demonstrates the advantage of blps over logical deduction and an approach based on mlns.
novel bayesian logic program implicit inform fact natur text experiment ic demonstr advantag blp logic deduct mln
2004.4 0.033736165593172836


In [51]:
#Debug
data['N13-2011']

['Helpfulness-Guided Review Summarization',
 ['Xiong, Wenting'],
 2013,
 'NAACL',
 ['C08-1103',
  'E09-1059',
  'P08-1031',
  'P11-2088',
  'D09-1032',
  'W12-3708',
  'C10-1039',
  'H05-1043',
  'N10-1122',
  'P11-1150',
  'P08-1036',
  'W06-1650',
  'E06-1039',
  'P12-1036'],
 ['CIn1', 'CIn2'],
 '\nreview mining and summarization has been a hot topic for the past decade.\na lot of effort has been devoted to aspect detection and sentiment analysis under the assumption that every review has the same utility for related tasks.\nhowever, reviews are not equally helpful as indicated by user-provided helpfulness assessment associated with the reviews.\nin this thesis, we propose a novel review summarization framework which summarizes review content under the supervision of automated assessment of review helpfulness.\nthis helpfulness-guided framework can be easily adapted to traditional review summarization tasks, for a wide range of domains.',
 '\n2 related work  the proposed work is grou

In [18]:
nltk.download()


showing info http://www.nltk.org/nltk_data/


True

In [41]:
sorted_scores = sorted(lta_scores, key=lambda x: x[1],reverse = True)

In [42]:
sorted_scores[0:10]

[('W07-0204', 0.7376442998215295),
 ('P09-2022', 0.3660613894842234),
 ('W12-2202', 0.3126140511356317),
 ('N09-3010', 0.12200302618107894),
 ('J05-2005', 0.10324274397827554),
 ('P09-1050', 0.03540509724253744),
 ('P08-3001', 0.024241383071877416),
 ('W02-1510', 0.022534243176556785),
 ('I05-3018', 0.021915779847156317)]

In [68]:
data['P11-2088']

['Automatically Predicting Peer-Review Helpfulness',
 ['Xiong, Wenting', ' Litman, Diane J.'],
 2011,
 'Proceedings of the 49th Annual Meeting of the Association for Computational Linguistics: Human Language Technologies',
 ['W06-1650', 'W09-3605', 'P05-1012', 'C00-1072'],
 ['N13-2011', 'W12-2020', 'W13-5008', 'W11-1402'],
 '\nidentifying peer-review helpfulness is an important task for improving the quality of feedback that students receive from their peers.\nas a first step towards enhancing existing peerreview systems with new functionality based on helpfulness detection, we examine whether standard product review analysis techniques also apply to our new context of peer reviews.\nin addition, we investigate the utility of incorporating additional specialized features tailored to peer review.\nour preliminary results show that the structural features, review unigrams and meta-data combined are useful in modeling the helpfulness of both peer reviews and product reviews, while peer-re